<a href="https://colab.research.google.com/github/yubars/ML-and-Cybersecurity-projects/blob/main/Distributed_Learning_using_CelebA_Image_dataset_along_with_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distributed Learning using CelebA Image dataset along with TPU
### Making use of TPU Strategy to implement clusters for distributed learning using TPU

### Learning Objectives
* Understand how Distributed learning works (using CNN)
* Learn the difference- between Distributed Learning and Federated Learning.
* Learn to use TPUStrategy for distributed learning.


### Imports
Import all the necessary libraries for the lab such as tensorflow, numpy, pandas, and keras.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from imutils import paths
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

### Mount Google Drive

In the code cell below, we mount the google drive to the colab environment so that we have access to the local version of the dataset.

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Read CSV
Use Pandas to load CelebA Image dataset from the CSV file.

In [ ]:
#read the csv file to access the feature "Smiling"
mydata = pd.read_csv('/content/drive/My Drive/Intro2MLDatasets/Lab9/list_attr_celeba.csv')

### Drop features
Drop the features from the dataset which are not needed. Smiling is the label to be predicted and the Image ID  is used as an identifier. All the other features are excluded from the dataset because in this lab we are reading each pixel of an image to be used as a feature and the Smiling attribute as a label. The prediction depends on the model which learns from the pixels of an image. 

In [ ]:
#drop all the other features from the csv except for the image_id and smiling
mydata.drop(mydata.columns.difference(['image_id','Smiling']), 1, inplace=True)
mydata.head(100)

,image_id,Smiling
0,000001.jpg,1
1,000002.jpg,1
2,000003.jpg,-1
3,000004.jpg,-1
4,000005.jpg,-1
...,...,...
95,000096.jpg,-1
96,000097.jpg,-1
97,000098.jpg,-1
98,000099.jpg,1


### Labels List
Create a list named Label which corresponds to the feature Smiling. This list will be used later to create training and testing data. 

In [ ]:
labels = list()

# Iterate over dataframe to store target labels for attribute "Smiling"
# Store all the attribute values for Smiling in labels 
for (columnName, columnData) in mydata.iteritems():
  if columnName == 'Smiling':
    for i in range(0, 2988):
      labels.append(columnData.values[i])

### Read image to convert into an array
Load function reads the images from the path provided as a list to the function. It reads the image, flattens the pixel value, and scales it to value [0,1] and returns the list of lists where each list is the scaled pixel value of one image.

In [ ]:
#Given a path, iterate over and read each image and convert to an array 
def load(path):
  data = list()

  for imgpath in path:
    im_grayscale= cv2.imread(imgpath, 0)
    image = np.array(im_grayscale)
    # .flatten()
    # image = image.reshape((218, 178, 1))
    #scales the image to [0,1] and adds to the list
    data.append(image/255)

  return data

### Image path
Create path for each image from the folder and call the load function. The length of image_list and labels are equal.

In [ ]:
image_path = '/content/drive/My Drive/Intro2MLDatasets/images_celeba'
img_paths = list(paths.list_images(image_path)) 

image_list = load(img_paths)

### Convert labels into binary and split data
Convert the label into binary format and split the data into train and test.

In [ ]:
#convert the label into binary data 0 or 1
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

#Split feature and label into train and test sets
x_train, x_test, y_train, y_test = train_test_split(image_list, labels, test_size=0.1, shuffle=True)

#convert all the variables to numpy array, and float32 type 
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

### Create Model
Create Neural network model with multiple layers. The functionality for each layers are defined below in the code cell.

https://www.kaggle.com/pavansanagapati/a-simple-cnn-model-beginner-guide

In [ ]:
def create_model(shape, classes):
  model = Sequential()
  """
  (218, 178, 3) is the shape of celebA image datasets.
  # Batch Normalization is used to make neural networks faster and more stable through normalization of layers inputs by recentering and rescaling. It maintains mean output close to 0 and the standard deviation close to 1.
  Conv2D is basically a 2D convolutional layer used for image processing kernel which helps produce a tensor of outputs.
  Maxpooling2D downsamples the input along its spatial dimension(height and width). The window is shifted by strides along each dimension.
  Dropout prevents overfitting. Fraction of input units to drop.
  Dense layer is the network layer in the model which feeds all outputs from the previous layer to all its neurons. Each neuron provides one output to the next layer.
  """

  model.add(Conv2D(64, input_shape=shape, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))

  model.add(Flatten())

  model.add(Dense(200, activation="relu"))
  model.add(Dropout(0.25))
  model.add(Dense(200, activation="relu"))
  model.add(Dropout(0.25))
  model.add(Dense(classes, activation="sigmoid"))

  return model

### TPU 
Using TPU strategy for synchronous training on TPUs.

In [ ]:
"""
tf.distribute.TPUStrategy lets you run your tf training on Tensor Processing Units (TPU). TPUs dramatically accelerates machine learning workloads.  
TPUStrategy implements synchronous distributed training. TPUs provide their own implementation of efficient all-reduce and other collective operations across multiple TPU cores.
"""
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
  model = create_model((218, 178, 1), 1)
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
      loss='binary_crossentropy',
      metrics=['accuracy'])

ValueError: ignored

In [ ]:
tf.get_logger().setLevel('ERROR')
validation_frequency = 5
history = model.fit(x_train, 
                    y_train, 
                    epochs=15,
                    steps_per_epoch=30,
                    validation_data=(x_test, y_test),
                    validation_freq=validation_frequency)

In [ ]:
def plot_training_history(history, title=None):
    fig = plt.figure(figsize=(9,9))
    plt.plot(history['accuracy'], label='acc')
    plt.plot(history['loss'], label='loss')
    plt.plot(history['val_accuracy'], label='val_acc')
    plt.plot(history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy and loss plot')
    plt.legend(loc='best')

In [ ]:
for metric in ["loss", "accuracy"]:
    metric_series = history.history[metric]
    print('break', history.history[metric])
    print(metric_series)

    temp = []
    for i in range(0, len(metric_series), validation_frequency):
      temp.append(metric_series[i])
    history.history[metric] = temp
    
    print('break2', history.history[metric])

plot_training_history(history.history)

 **Assignment**

**We have ML scenario with 100 devices which could use Federated Learning (FL) or Distributed Learning (DL). Let's assume that each device generates 25GB of data every hour. Let's consider that the DL and FL training process takes more or less same time (say approx. 20 minutes). For this scenario, answer the following questions:**
  
1. **In Distributed Learning, how long would it take to exchange this data using 100 Mbps communication link if we are updating the learning model every hour? Approximately how long would it take to get the learning model ready for inferencing or prediction for each device?**

Assuming parallel exhanges between all nodes, 

Time taken to exchange 25GB hourly data in 100 Mbps link = (25GB /100 Mbps)/60 min
=(25 x 1024 x 8 )/(100 x 60) Mins
=34.13 mins

Time taken to get learning model ready for infeencing  = 34.13 min + 20 min training time
                      = 54.13 mins
2. **In Federated Learning, how long would it take to exchange 100 parameters(each parameter is 1KBytes) from each client to FL server using 100 Mbps communication link if we are updating the learning model every hour and sending the global model back to the client? Let's assume that FL server takes about 2 mins for generating global model. Approximately how long would it take to get the learning model ready for inferencing or prediction for each device?**

Time taken to get learning model ready for prediction 
= 20 min (device training time) + 2 min (global  model generation time) + 2 x (100 x 1 x 8)/(100 x 1024 x 60) = 22.00026 Min


3. **Based on 1 and 2, provide your observations on DL and FL.**

Based of 1 and 2, I observed that Federated learnig is far better than distributed learning in terms of time performance. Additionally, Federated learning perserves user privacy and enhances security by not sharing data to others as it only sends parameters to global server only. However, if we consider accuracy, DL might perform better than FL.